<a href="https://colab.research.google.com/github/lauxpress/NLP_UBA_Scheihing23/blob/main/NLP_Desaf%C3%ADo_3_Scheihing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word embedding
Vamos a realizar un custom embedding con la libreria Gensim

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import multiprocessing
from gensim.models import Word2Vec
import re # Regular Expressions (regex)

In [ ]:
# Cargar el modelo en inglés
nlp = spacy.load('en_core_web_sm')

# Leer el archivo de texto
with open('/content/ontheroad_cap1.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Substituir
text = re.sub(r'«', '»',text)

In [ ]:
# Procesar el texto con spaCy para obtener las oraciones
doc = nlp(text)
sentences = [sent.text for sent in doc.sents]

# Crear un DataFrame con las oraciones
df = pd.DataFrame(sentences, columns=['Oraciones'])
df

,Oraciones
0,I first met Dean not long after my wife and I ...
1,I had just gotten over a serious illness that ...
2,With the coming of Dean Moriarty began the par...
3,Before that I’d often dreamed of going West to...
4,Dean is the perfect guy for the road because h...
...,...
370,"Well, I can take you a hundred miles up the line."
371,"»\n»Grand, grand, you saved my life."
372,"»\n»I used to hitchhike myself, that’s why I a..."
373,»\n »I would too if I had a car.»


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 375


Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
#Ahora todas las oraciones estan separadas en tokens
sentence_tokens[:3]

[['i',
  'first',
  'met',
  'dean',
  'not',
  'long',
  'after',
  'my',
  'wife',
  'and',
  'i',
  'split',
  'up'],
 ['i',
  'had',
  'just',
  'gotten',
  'over',
  'a',
  'serious',
  'illness',
  'that',
  'i',
  'won’t',
  'bother',
  'to',
  'talk',
  'about',
  'except',
  'that',
  'it',
  'had',
  'something',
  'to',
  'do',
  'with',
  'the',
  'miserably',
  'weary',
  'split',
  'up',
  'and',
  'my',
  'feeling',
  'that',
  'everything',
  'was',
  'dead'],
 ['with',
  'the',
  'coming',
  'of',
  'dean',
  'moriarty',
  'began',
  'the',
  'part',
  'of',
  'my',
  'life',
  'you',
  'could',
  'call',
  'my',
  'life',
  'on',
  'the',
  'road']]

Creamos los vectores

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de docs en el corpus: 375
Cantidad de words distintas en el corpus: 271


Entrenamos los embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 75361.6015625
Loss after epoch 1: 33497.7109375
Loss after epoch 2: 33897.125
Loss after epoch 3: 33203.734375
Loss after epoch 4: 34184.546875
Loss after epoch 5: 34151.9375
Loss after epoch 6: 33431.25
Loss after epoch 7: 33551.65625
Loss after epoch 8: 33200.21875
Loss after epoch 9: 33831.15625
Loss after epoch 10: 33780.71875
Loss after epoch 11: 33000.21875
Loss after epoch 12: 33372.4375
Loss after epoch 13: 33800.03125
Loss after epoch 14: 32847.90625
Loss after epoch 15: 33219.1875
Loss after epoch 16: 34056.625
Loss after epoch 17: 33395.875
Loss after epoch 18: 33906.4375
Loss after epoch 19: 33138.75


(65129, 166820)

Ensayos:

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dean"], topn=10)

[('chad', 0.9989824891090393),
 ('everybody', 0.9988863468170166),
 ('its', 0.9988683462142944),
 ('every', 0.9988682866096497),
 ('beautiful', 0.998860776424408),
 ('god', 0.9988541007041931),
 ('rain', 0.998846709728241),
 ('middle', 0.9988402128219604),
 ('gray', 0.9988251328468323),
 ('day', 0.9988231658935547)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["dean"], topn=10)

[('york', -0.9919135570526123),
 ('new', -0.9934605956077576),
 ('know', -0.9940288662910461),
 ('to', -0.9947744011878967),
 ('as', -0.9952324032783508),
 ('didn’t', -0.9956567883491516),
 ('would', -0.9957091212272644),
 ('could', -0.996131181716919),
 ('an', -0.9962911009788513),
 ('be', -0.9964403510093689)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["mad"], topn=10)

[('are', 0.9989535808563232),
 ('people', 0.9988415241241455),
 ('guy', 0.9987956285476685),
 ('who', 0.9987594485282898),
 ('never', 0.9987443089485168),
 ('talk', 0.9987119436264038),
 ('too', 0.9986883401870728),
 ('even', 0.9986675977706909),
 ('money', 0.998623788356781),
 ('river', 0.9986048936843872)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["mad"], topn=10)

[('york', -0.9911127686500549),
 ('new', -0.9921311140060425),
 ('as', -0.9949281215667725),
 ('the', -0.9950131773948669),
 ('long', -0.9953908920288086),
 ('an', -0.9954916834831238),
 ('could', -0.9959725141525269),
 ('know', -0.9959810376167297),
 ('in', -0.9959972500801086),
 ('into', -0.9960350394248962)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["car"], topn=10)

[('hour', 0.998596727848053),
 ('out', 0.9985631704330444),
 ('sitting', 0.9985014796257019),
 ('half', 0.9984301924705505),
 ('davenport', 0.998424232006073),
 ('under', 0.9983940720558167),
 ('drove', 0.998393714427948),
 ('station', 0.9983934164047241),
 ('clear', 0.998387336730957),
 ('dollars', 0.9983595013618469)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["road"], topn=10)

[('america', 0.9988267421722412),
 ('middle', 0.9987679719924927),
 ('everybody', 0.9987400770187378),
 ('sun', 0.9987112879753113),
 ('railroad', 0.9986905455589294),
 ('rain', 0.9986276030540466),
 ('till', 0.9986093640327454),
 ('shelton', 0.9986037611961365),
 ('same', 0.9985983371734619),
 ('day', 0.9985952973365784)]

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

No se si se puede ver en el segundo grafico (si es que quedo hecho el acercamiento que realice) pero este detalle me parecio muy interesante y me gusto mucho ya qeu es parte de una de las frases que mas me gustan de este libro y es la de "I shambled after as I’ve been doing all my life
after people who interest me, because the only people for me are the mad ones, the ones who are mad to live, mad to talk, mad to be saved, desirous of everything at the same time..."
Por esa razon podemos observar los embeddings de las palabras -People, Who, Mad- todas juntas en proximidad.
Tambien notamos proximidad entre las palabras New y York por razones obvias y casi todas las palabras qeu se puedan relacionar con "car" se encuentran cerca de la misma.

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

En el grafico 3D notamos que las palabras relacionadas a los dias, como son "hours" "day" "long". Sabemos que en esencia la distribucion quedo correcta y a pesar de que no encontre ciertas proximidades que esperaba como por ejemplo "carlo" y "marx" estan bastante alejadas y no cerca como se esperaria siendo que es el nombre de un personaje "carlo marx" pero quiza eso se deba a que la contextualizacion de ambas palabras terminan separandolas.